In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('nsmc-master/ratings_train.txt', delimiter='\t',
                      keep_default_na=False)
df_train.head(n=3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
text_train = df_train['document'].as_matrix()
y_train = df_train['label'].as_matrix()

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [4]:
df_test = pd.read_csv('nsmc-master/ratings_test.txt', delimiter='\t',
                      keep_default_na=False)
text_test = df_test['document'].as_matrix()
y_test = df_test['label'].as_matrix()

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [5]:
from konlpy.tag import Twitter
twitter_tag = Twitter()

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [6]:
def twitter_tokenizer(text):
    return twitter_tag.morphs(text)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

twit_param_grid = {'tfidfvectorizer__min_df':[3,5,7],
                    'tfidfvectorizer__ngram_range': [(1,1),(1,2),(1,3)],
                    'logisticregression__C':[0.1,1,10,100]}
twit_pipe = make_pipeline(TfidfVectorizer(tokenizer=twitter_tokenizer),
                          LogisticRegression())
twit_grid = GridSearchCV(twit_pipe, twit_param_grid)

twit_grid.fit(text_train,y_train)

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tfidfvectorizer__min_df': [3, 5, 7], 'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)], 'logisticregression__C': [0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [40]:
import MeCab
mecab = MeCab.Tagger()
def mecab_tokenizer(text):
    return mecab.morphs(text)

In [42]:
mecab_param_grid = {'tfidfvectorizer__min_df':[3,5,7],
                    'tfidfvectorizer__ngram_range': [(1,1),(1,2),(1,3)],
                    'logisticregression__C':[0.1,1,10,100]}

In [45]:
mecab_pipe = make_pipeline(TfidfVectorizer(tokenizer=mecab_tokenizer),
                          LogisticRegression())
mecab_grid = GridSearchCV(mecab_pipe, mecab_param_grid, n_jobs=-1)

In [46]:
mecab_grid.fit(text_train,y_train)

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


PicklingError: Could not pickle the task to send it to the workers.

In [8]:
twit_grid.best_score_

0.8583866666666666

In [53]:
X_test_konlpy = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)

In [60]:
twit_grid.best_estimator_.named_steps["logisticregression"].score(X_test_konlpy, y_test)

0.86506

In [64]:
twit_grid.best_estimator_.named_steps["logisticregression"].predict(X_test_konlpy)

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [52]:
money_test = pd.read_csv('돈.txt', delimiter='\t',keep_default_na=False)
text_test = money_test['document'].as_matrix()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb5 in position 0: invalid start byte

In [9]:
import json
js = json.load(open('계룡산_naver_blog_fixed.json', encoding='utf-8'))

In [30]:
text = ''
for i in range(0,len(js)):
    try:
        text = text+js[i]['doc_content']
    except:
        pass

In [38]:
text.split('\n').remove(' ')

In [41]:
text2 = text.split('\n')

In [42]:
text2.remove(' ')

In [47]:
text3 = []
for i in range(0,len(text2)):
    if text2[i] != '' and text2[i] != ' ':
        text3.append(text2[i])
    

In [53]:
import pandas
df = pd.DataFrame(text3)

In [59]:
test = df[0].as_matrix()

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [50]:
text4 = text3.as_matrix()

AttributeError: 'list' object has no attribute 'as_matrix'

In [67]:
text_test

array(['시간가는줄모르고봤네요ㅎㅎ',
       '박누리 감독 데뷔작인데 생각보다 훨씬 탄탄하다. 이야기 자체가 엄청 새롭거나 심오하거나 하지는 않지만 가지고 있는 재료 안에서 충분히 맛을 냈다. 카메라 워킹 훌륭하고 음악도 좋다. 무엇보다 한국영화계의 믿보배로 성장한 류준열 연기를 맘껏 볼 수 있다',
       '쉼표 3개에 0이 10개면 100억이다!!', ..., '되도않는 류준열의 디카프리오 따라하기',
       '관람객그럭저럭 볼만함 두시간 내내 지루하지 않았던게 크고 쫄리는 부분은 잘 쫄리도록 음향감독님이 굉장히 열일(?)하신듯한 영화... 배우 연기들도 괜찮았고 무엇보다 유지태아저씨 너무섹시함',
       '몰입도 좋고 진행 매끄럽고 다 좋은데 영화끝나고 곰곰히 생각해보니 헛점도 몇개있고 엔딩도 ‘그래서 조일현은 어케된거임?’ 했음완전 깔끔하고 명쾌하지 않지만 한번, 볼만은 함'],
      dtype=object)

In [68]:
money_test_konlpy = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)

In [60]:
test_konlpy = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(test)

In [70]:
label = twit_grid.best_estimator_.named_steps["logisticregression"].predict(money_test_konlpy)

In [61]:
label1 = twit_grid.best_estimator_.named_steps["logisticregression"].predict(test_konlpy)

In [64]:
len(label1)

14260

In [71]:
len(label)

1825

In [76]:
from pandas import DataFrame, Series

In [67]:
money_df = pd.DataFrame(text_test,columns=['document'])

In [68]:
test_df = pd.DataFrame(test,columns=['document'])
temp = pd.Series(label1, name='label')
test_df = test_df.join(temp)

In [70]:
test_df.to_excel('계룡산감정분석.xlsx')

In [85]:
temp = pd.Series(label, name='label')

In [87]:
money_df = money_df.join(temp)

ValueError: columns overlap but no suffix specified: Index(['label'], dtype='object')

In [72]:
test_df

,document,label
0,"안녕하세요 계룡산자락에서 농사짓는""풀""입니다.이웃과 하는 쌈박스 이번엔 ""꿀""과 ""...",0
1,많은 꿀들이 시중에 유통되고 있고품질도 제각각이라 어떤 꿀을 선택해야 할지고민이 되...,1
2,남편의 커피에는 설탕 대신 계룡산 꿀을 넣어 주고 있어요,1
3,첫째. 청정 계룡산국립공원에서 채밀!대부분의 양봉업자들은 꽃을 찾아 전국을 돌아다니...,1
4,"포장단위는 500그람씩 소포장과2.4킬로 선물용 포장이 있습니다.♥가격, 용량500...",1
5,"♥가격/용량화분230 그람 14,000원(배송비2500별도)화분1키로 60,000원...",1
6,국립공원에서 운영하는 캠핑장은 매달 1일과 15일에 15일 이후의 일정을 보름씩 끊...,1
7,공주 어느 작은 읍내에 위치하였는데 생활의달인 덕분에 대박집 느낌이 물씬-대기실도 ...,1
8,우리 가족 발걸음 제일 빠른 사람-캠핑을 갈꺼라 복장이 좀 집에서 막 나온사람같아....,1
9,중국집인테 특이하게 공주 장순루는 겉절이 김치를 주더라..그런데 이 김치가~~ 칼국...,0


In [90]:
writer = pd.ExcelWriter('돈.xlsx')
money_df.to_excel(writer,'Sheet1')
writer.save()

In [138]:
from pandas import DataFrame, Series

def extract_opinion(title):
    df_test = pd.read_csv('review/'+title+'.txt', delimiter='\t',
                      keep_default_na=False,header=None)
    text_test = df_test[0].as_matrix()
    test_konlpy = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
    label = twit_grid.best_estimator_.named_steps["logisticregression"].predict(test_konlpy)
    df = DataFrame(text_test,columns=['document'])
    temp = pd.Series(label, name='label')
    df = df.join(temp)
    writer = pd.ExcelWriter('positive_negative/'+title+'.xlsx')
    df.to_excel(writer,'Sheet1')
    writer.save() 

In [118]:
extract_opinion('가버나움')

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [110]:
df_test = pd.read_csv('review/PMC.txt', delimiter='\t',
                      keep_default_na=False, header=None)

In [142]:
movie_name = open('./영화제목.txt',encoding = 'utf8')
movie_title = [num for num in movie_name.read().split('\n')]
movie_title

['아쿠아맨',
 '보헤미안 랩소디',
 'PMC',
 '글래스',
 '극장판 공룡메카드',
 '범블비',
 '뺑반',
 '러브 유어셀프 인 서울',
 '그린 북',
 '스윙키즈',
 '드래곤 길들이기 3',
 '언니',
 '언더독',
 '점박이 한반도의 공룡2',
 '미래의 미라이',
 '구스 베이비',
 '그대 이름은 장미',
 '극장판 짱구는 못말려',
 '스파이더맨',
 '그린치',
 '극장판 헬로카봇',
 '마약왕',
 '알리타',
 '증인',
 '사바하',
 '해피 데스데이 2 유',
 '기묘한 가족',
 '메리 포핀스 리턴즈',
 '신데렐라',
 '항거',
 '명탐정 코난',
 '콜드 체이싱',
 '가버나움',
 '레고 무비2',
 '자전차왕 엄복동',
 '캡틴 마블',
 '돈',
 '어스',
 '이스케이프 룸',
 '악질경찰',
 '덤보',
 '우상',
 '장난스런 키스',
 '라스트 미션',
 '더 페이버릿',
 '1919 유관순',
 '어쩌다, 결혼',
 '어벤져스',
 '생일',
 '샤잠!',
 '헬보이',
 '뽀로로 극장판 보물섬 대모험',
 '미성년',
 '요로나의 저주',
 '바이스',
 '공포의 묘지',
 '크게 될 놈',
 '아이 엠 마더',
 '파이브 피트',
 '로망',
 '왓칭',
 '나의 특별한 형제',
 '콜레트',
 '다시, 봄',
 '노무현과 바보들',
 '악인전',
 '걸캅스',
 '알라딘',
 '기생충',
 '명탐정 피카츄',
 '배심원들',
 '어린 의뢰인',
 '고질라: 킹 오브 몬스터',
 '더 보이',
 '호텔 뭄바이',
 '0.0MHz',
 '교회오빠',
 '어글리 돌',
 '서스페리아',
 '안도 타다오',
 '판의 미로 - 오필리아와 세 개의 열쇠',
 '미스 스티븐스',
 '토이 스토리 4',
 '엑스맨',
 '맨 인 블랙',
 '존 윅 3',
 '애나벨 집으로',
 '천로역정: 천국을 찾아서',
 '비스트',
 '로켓맨',
 '사탄의 인형',
 '빅샤크2',
 '업사이드',
 '세상을 바꾼 변호인',

In [144]:
for i in range(len(movie_title)):
    try:
        extract_opinion(movie_title[i])
    except:
        print(movie_title[i])
        continue

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


언니
점박이 한반도의 공룡2
극장판 헬로카봇
고질라: 킹 오브 몬스터
토이 스토리 4
천로역정: 천국을 찾아서



In [147]:
def extract_opinion2():
    df_test = pd.read_csv('review/언니.txt', delimiter='\t',
                      keep_default_na=False,header=None)
    text_test = df_test[0].as_matrix()
    test_konlpy = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
    label = twit_grid.best_estimator_.named_steps["logisticregression"].predict(test_konlpy)
    df = DataFrame(text_test,columns=['document'])
    temp = pd.Series(label, name='label')
    df = df.join(temp)
    writer = pd.ExcelWriter('positive_negative/언니.xlsx')
    df.to_excel(writer,'Sheet1')
    writer.save() 

In [164]:
extract_opinion('극장판 헬로카봇')

C:\Users\CPB06GameN\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
